# Get frequency count for transcription text

In [1]:
# !pip install nltk==3.5

In [30]:
import re
import pandas as pd
import nltk
import json

In [3]:
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

from nltk import FreqDist

nltk.download('punkt')
nltk.download('wordnet')

import string
punctuation = set(string.punctuation)
stop_words_with_punct = stop_words.union(punctuation)

# from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
# from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oliviakasmin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oliviakasmin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oliviakasmin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
stop_words_with_punct.add("’")
stop_words_with_punct.add("“")
stop_words_with_punct.add("⎬")
stop_words_with_punct.add("┃")
stop_words_with_punct.add("|")
stop_words_with_punct.add("●")
stop_words_with_punct.add("--")
stop_words_with_punct.add("--")

In [5]:
df_full = pd.read_parquet('../nara_pension_file_pages_with_file_cat.parquet')

In [6]:
df_full.head()

,NAID,naraURL,title,logicalDate,variantControlNumbers,pdfObjectID,pdfURL,pageObjectId,pageURL,pageImageType,ocrID,ocrText,ocrUploadDate,ocrContributor,transcriptionID,transcriptionText,transcriptionContributionCount,transcriptionUserNames,transcriptionDate,file_cat
0,111769430,https://catalog.archives.gov/id/111769430,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769431,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850379,Mass\nWoodbury\nWm\nS33948,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,soldier
1,111769430,https://catalog.archives.gov/id/111769430,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769432,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850380,33948\nRevy INVALID.\nFile No. 33948\nIndex\nW...,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,soldier
2,111769430,https://catalog.archives.gov/id/111769430,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769433,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850381,To the Secretary of the Department of War\nSir...,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,soldier
3,111769418,https://catalog.archives.gov/id/111769418,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769419,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850368,"(3 - 1776)\nSERVICE\nMass\nCONTENTS\nWoodbury,...",2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,soldier
4,111769418,https://catalog.archives.gov/id/111769418,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769420,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850369,1831\nto\nSchaled Tucker\n1941\nMassachusetts\...,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,soldier


In [7]:
# Filter to rows that contain transcriptionText
df = df_full[df_full['transcriptionText'].notna()]

In [8]:
print(f"Filtered dataframe shape: {df.shape}")
print(f"Number of rows with transcriptionText: {len(df)}")

Filtered dataframe shape: (604716, 20)
Number of rows with transcriptionText: 604716


In [9]:
df.head()

,NAID,naraURL,title,logicalDate,variantControlNumbers,pdfObjectID,pdfURL,pageObjectId,pageURL,pageImageType,ocrID,ocrText,ocrUploadDate,ocrContributor,transcriptionID,transcriptionText,transcriptionContributionCount,transcriptionUserNames,transcriptionDate,file_cat
14,111769405,https://catalog.archives.gov/id/111769405,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769406,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850356,(3 - 1776)\nSERVICE\nMass.\nCONTENTS\nWoodbury...,2024-11-20T20:32:03.000Z,FamilySearch,115c69ad-67bb-47c3-8c93-36b92f73483a,"(3 - 1776)\nSERVICE\nMass.\nWoodbury, Peter\n1...",1.0,carlaMSUspartan,2025-07-27 18:31:27,soldier
15,111769405,https://catalog.archives.gov/id/111769405,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769407,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850357,Vermont 18.682\n---\nPeter Woodbury\nof Windso...,2024-11-20T20:32:03.000Z,FamilySearch,3fc8f1d1-c8be-468a-9d45-6183c4076266,Vermont 18.682\n---\nPeter Woodbury\nof Windso...,1.0,carlaMSUspartan,2025-07-27 18:31:59,soldier
16,111769405,https://catalog.archives.gov/id/111769405,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769408,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850358,9316\nPeter Woodbery\nAdmit\nis 28\nReturn the...,2024-11-20T20:32:03.000Z,FamilySearch,29da1ecc-f3ee-413c-bdb7-9811a730c9af,9316\nPeter Woodbury\nAdmit\n11 mos 28 ds\nRet...,1.0,carlaMSUspartan,2025-07-27 18:36:51,soldier
17,111769405,https://catalog.archives.gov/id/111769405,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769409,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850359,BRIEF objections to the admission of Pension C...,2024-11-20T20:32:03.000Z,FamilySearch,d781857a-6715-4d09-82f0-98ba8318a62f,BRIEF objections to the admission of Pension C...,1.0,carlaMSUspartan,2025-07-27 18:37:48,soldier
18,111769405,https://catalog.archives.gov/id/111769405,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769410,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850360,XI. Has the applicant obtained the evidence of...,2024-11-20T20:32:03.000Z,FamilySearch,4b9ecfef-8f54-49af-8cfe-3dae2b1ff530,2\nXI. Has the applicant obtained the evidence...,1.0,carlaMSUspartan,2025-07-27 18:40:15,soldier


### Clean transcriptionText

1. change "\n" to " "
2. tokenize words
3. filter to remove stop words

In [10]:
sample = df.iloc[2000]['transcriptionText']
sample_str = sample.replace("\n", " ").lower()

In [11]:
sample_str

'pension office feb 25- washington d.c. did william withers who enlisted in the revolutionary war in 1777. but was not called till 1778. get a pension and please tell me why he did not get one. said wm withers was in grants co. and grayson troops was commissioned a sergeant, fell ill at valley forge. and was sent to the country where he received his pay- an early reply will be much appreciated by mrs ada withers bunger. - west point ky feby 25th 1935.  [illegible] wm. withers  [stamp] received feb 27 1935'

In [12]:
# First tokenize the text into words
words = word_tokenize(sample_str)
# words

# Then filter out stop words
meaningful_words = [word for word in words if word.casefold() not in stop_words_with_punct]
# meaningful_words

# Then get frequency distribution
frequency_distribution = FreqDist(meaningful_words)

In [13]:
frequency_distribution

FreqDist({'withers': 4, 'pension': 2, 'feb': 2, 'get': 2, 'wm': 2, 'received': 2, '1935': 2, 'office': 1, '25-': 1, 'washington': 1, ...})

In [14]:
def get_frequency_distribution(text):
    sample_str = text.replace("\n", " ").lower()
    words = word_tokenize(sample_str)
    meaningful_words = [word for word in words if word.casefold() not in stop_words_with_punct]
    frequency_distribution = FreqDist(meaningful_words)
    freq_dict = dict(frequency_distribution)
    # sorted_freq_dict = dict(sorted(freq_dict.items(), key=lambda x: x[1], reverse=True))
    return freq_dict

In [15]:
result = get_frequency_distribution(sample)
result

{'pension': 2,
 'office': 1,
 'feb': 2,
 '25-': 1,
 'washington': 1,
 'd.c.': 1,
 'william': 1,
 'withers': 4,
 'enlisted': 1,
 'revolutionary': 1,
 'war': 1,
 '1777.': 1,
 'called': 1,
 'till': 1,
 '1778.': 1,
 'get': 2,
 'please': 1,
 'tell': 1,
 'one': 1,
 'said': 1,
 'wm': 2,
 'grants': 1,
 'co.': 1,
 'grayson': 1,
 'troops': 1,
 'commissioned': 1,
 'sergeant': 1,
 'fell': 1,
 'ill': 1,
 'valley': 1,
 'forge': 1,
 'sent': 1,
 'country': 1,
 'received': 2,
 'pay-': 1,
 'early': 1,
 'reply': 1,
 'much': 1,
 'appreciated': 1,
 'mrs': 1,
 'ada': 1,
 'bunger': 1,
 'west': 1,
 'point': 1,
 'ky': 1,
 'feby': 1,
 '25th': 1,
 '1935': 2,
 'illegible': 1,
 'stamp': 1,
 '27': 1}

### Get frequency count for each row

In [16]:
df_with_dict = df[['NAID', 'naraURL', 'pageObjectId', 'pageURL', 'transcriptionText', 'file_cat']].copy()

# cols: NAID, naraURL, pageObjectId, pageURL, transcriptionText, file_cat

In [17]:
df_with_dict.head()

,NAID,naraURL,pageObjectId,pageURL,transcriptionText,file_cat
14,111769405,https://catalog.archives.gov/id/111769405,111769406,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,"(3 - 1776)\nSERVICE\nMass.\nWoodbury, Peter\n1...",soldier
15,111769405,https://catalog.archives.gov/id/111769405,111769407,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Vermont 18.682\n---\nPeter Woodbury\nof Windso...,soldier
16,111769405,https://catalog.archives.gov/id/111769405,111769408,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,9316\nPeter Woodbury\nAdmit\n11 mos 28 ds\nRet...,soldier
17,111769405,https://catalog.archives.gov/id/111769405,111769409,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,BRIEF objections to the admission of Pension C...,soldier
18,111769405,https://catalog.archives.gov/id/111769405,111769410,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,2\nXI. Has the applicant obtained the evidence...,soldier


In [18]:
df_with_dict['frequency_distribution'] = df_with_dict['transcriptionText'].apply(get_frequency_distribution)
df_with_dict['frequency_keys'] = df_with_dict['frequency_distribution'].apply(lambda x: '||'.join(x.keys()))

In [21]:
df_with_dict.head()

,NAID,naraURL,pageObjectId,pageURL,transcriptionText,file_cat,frequency_distribution,frequency_keys
14,111769405,https://catalog.archives.gov/id/111769405,111769406,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,"(3 - 1776)\nSERVICE\nMass.\nWoodbury, Peter\n1...",soldier,"{'3': 1, '1776': 1, 'service': 1, 'mass': 1, '...",3||1776||service||mass||woodbury||peter||10009...
15,111769405,https://catalog.archives.gov/id/111769405,111769407,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Vermont 18.682\n---\nPeter Woodbury\nof Windso...,soldier,"{'vermont': 3, '18.682': 1, '--': 1, 'peter': ...",vermont||18.682||--||peter||woodbury||windsor|...
16,111769405,https://catalog.archives.gov/id/111769405,111769408,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,9316\nPeter Woodbury\nAdmit\n11 mos 28 ds\nRet...,soldier,"{'9316': 1, 'peter': 1, 'woodbury': 1, 'admit'...",9316||peter||woodbury||admit||11||mos||28||ds|...
17,111769405,https://catalog.archives.gov/id/111769405,111769409,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,BRIEF objections to the admission of Pension C...,soldier,"{'brief': 1, 'objections': 1, 'admission': 1, ...",brief||objections||admission||pension||claims|...
18,111769405,https://catalog.archives.gov/id/111769405,111769410,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,2\nXI. Has the applicant obtained the evidence...,soldier,"{'2': 1, 'xi': 1, 'applicant': 5, 'obtained': ...",2||xi||applicant||obtained||evidence||clergyma...


In [20]:
# df_with_dict.to_parquet('df_transcriptionText_with_frequency_dict.parquet')

### Combine frequency count for each row

In [22]:
from collections import Counter

# Combine all frequency distributions
combined_freq = Counter()
for freq_dict in df_with_dict['frequency_distribution']:
    combined_freq.update(freq_dict)

# Convert to regular dictionary if needed
combined_freq_dict = dict(combined_freq)

In [23]:
combined_freq_dict

{'3': 70601,
 '1776': 41369,
 'service': 357213,
 'mass': 19773,
 'woodbury': 1933,
 'peter': 24023,
 '10009': 3,
 'number': 35125,
 's.': 61188,
 '19171': 4,
 'carded': 6106,
 'contents': 12150,
 'vermont': 46238,
 '18.682': 3,
 '--': 254938,
 'windsor': 8720,
 'co': 23990,
 'state': 476444,
 'private': 97743,
 'compy': 2563,
 'commanded': 82831,
 'captain': 118973,
 'wilder': 857,
 'regt': 24635,
 'col.': 53994,
 'dolittle': 19,
 'line': 78503,
 '11': 19662,
 'mo': 16958,
 '28': 16565,
 'days': 46289,
 'inscribed': 27395,
 'roll': 67871,
 'rate': 44672,
 '39': 1683,
 'dollars': 58168,
 '74': 2279,
 'cents': 31212,
 'per': 66189,
 'annum': 44492,
 'commence': 27152,
 '4th': 75100,
 'day': 398882,
 'march': 173235,
 '1831.': 7302,
 'certificate': 146624,
 'pension': 372518,
 'issued': 47514,
 '25': 19645,
 'sept': 30145,
 '1833': 38272,
 'sent': 50141,
 'oel': 491,
 'billings': 1542,
 'royalton': 1169,
 'vt': 5609,
 'arrears': 21450,
 "'33": 1354,
 '99.35': 4,
 'semi-anl': 9261,
 'allo

#### remove any words with fewer than 100 entries

In [24]:
# Filter to keep only words with 100 or more occurrences
filtered_combined_freq_dict = {word: count for word, count in combined_freq_dict.items() if count >= 100}

In [26]:
len(filtered_combined_freq_dict)

14800

In [27]:
# Sort by frequency count (descending - most frequent first)
sorted_filtered_combined_freq_dict = dict(sorted(filtered_combined_freq_dict.items(), key=lambda x: x[1], reverse=True))

In [28]:
sorted_filtered_combined_freq_dict

{'county': 664887,
 'said': 651461,
 'state': 476444,
 'day': 398882,
 'pension': 372518,
 'court': 363693,
 'service': 357213,
 'war': 297828,
 'illegible': 280451,
 '--': 254938,
 'john': 253109,
 'served': 243838,
 'act': 223764,
 'years': 219799,
 'one': 211716,
 'time': 210263,
 'year': 203191,
 'revolutionary': 203008,
 'sworn': 197806,
 'new': 192953,
 'certify': 188545,
 'office': 176854,
 'declaration': 174458,
 'march': 173235,
 "'s": 162879,
 'company': 161326,
 'june': 157751,
 'may': 157376,
 'clerk': 153802,
 'aforesaid': 148902,
 'months': 148143,
 'certificate': 146624,
 'seal': 142696,
 'made': 142230,
 'states': 141356,
 'widow': 138401,
 '``': 138029,
 'claim': 136181,
 'name': 135890,
 'justice': 135485,
 'subscribed': 134538,
 'soldier': 132836,
 'hereby': 130678,
 'peace': 128527,
 'july': 125974,
 'william': 125130,
 'captain': 118973,
 'york': 116356,
 'record': 115630,
 'james': 113545,
 'first': 112217,
 'hand': 111877,
 'washington': 110831,
 'regiment': 1107

In [ ]:
# Save the sorted filtered dictionary to JSON
with open('sorted_filtered_combined_freq_dict.json', 'w') as f:
    json.dump(sorted_filtered_combined_freq_dict, f, indent=2)

### Save smaller df with just rows from animals, hopelessness, and religious categories

In [31]:
# Load the hopelessness words from JSON
with open('frequency_hopelessness_10.json', 'r') as f:
    hopelessness_dict = json.load(f)

# Extract just the keys (words)
hopelessness_words = list(hopelessness_dict.keys())

# Display the words
# print(f"Imported {len(hopelessness_words)} hopelessness words:")
# print(hopelessness_words)

Imported 15 hopelessness words:
['needy', 'poverty', 'destitute', 'afflicted', 'affliction', 'helpless', 'suffering', 'suffer', 'pain', 'painful', 'hardships', 'hardship', 'distress', 'distressed', 'despondent']


In [33]:
with open('frequency_religion_10.json', 'r') as f:
    religion_dict = json.load(f)

# Extract just the keys (words)
religion_words = list(religion_dict.keys())

# Display the words
# print(f"Imported {len(religion_words)} religion words:")
# print(religion_words)

Imported 10 religion words:
['clergyman', 'lord', 'church', 'bible', 'christian', 'minister', 'god', 'bishop', 'pastor', 'pope']


In [34]:
with open('frequency_animals_farm_10.json', 'r') as f:
    animals_dict = json.load(f)

# Extract just the keys (words)
animals_words = list(animals_dict.keys())

# Display the words
# print(f"Imported {len(animals_words)} animals words:")
# print(animals_words)

Imported 15 animals words:
['cow', 'cows', 'bull', 'cattle', 'sheep', 'lamb', 'lambs', 'hogs', 'hog', 'pigs', 'pig', 'mare', 'colt', 'heifer', 'heifers']


In [35]:
def categorize_frequency_keys(frequency_keys_str):
    """
    Categorize frequency keys based on word lists.
    Returns a string of categories separated by '||'
    """
    if pd.isna(frequency_keys_str) or frequency_keys_str == '':
        return ''
    
    # Split the frequency_keys string into individual words
    words = set(frequency_keys_str.split('||'))
    
    categories = []
    
    # Check for each category
    if words.intersection(hopelessness_words):
        categories.append('hopelessness')
    
    if words.intersection(religion_words):
        categories.append('religion')
    
    if words.intersection(animals_words):
        categories.append('animals')
    
    # Join categories with '||' if multiple, return empty string if none
    return '||'.join(categories) if categories else None

In [36]:
# Apply the function to create the new column
df_with_dict_categorized = df_with_dict.copy()
df_with_dict_categorized['frequency_categories'] = df_with_dict_categorized['frequency_keys'].apply(categorize_frequency_keys)

In [38]:
# Check the results
print(f"Total rows: {len(df_with_dict_categorized)}")
print(f"\nCategory distribution:")
print(df_with_dict_categorized['frequency_categories'].value_counts())
print(f"\nSample rows with categories:")
print(df_with_dict_categorized[['frequency_keys', 'frequency_categories']].head(20))

Total rows: 604716

Category distribution:
frequency_categories
religion                           56037
animals                             6229
hopelessness                        5477
religion||animals                    842
hopelessness||religion               812
hopelessness||animals                437
                                      66
hopelessness||religion||animals       58
Name: count, dtype: int64

Sample rows with categories:
                                       frequency_keys frequency_categories
14  3||1776||service||mass||woodbury||peter||10009...                 None
15  vermont||18.682||--||peter||woodbury||windsor|...                 None
16  9316||peter||woodbury||admit||11||mos||28||ds|...                 None
17  brief||objections||admission||pension||claims|...                 None
18  2||xi||applicant||obtained||evidence||clergyma...             religion
19  3||important||cases||determine||precision||per...                 None
20  9316||peter||woodbury||

In [39]:
df_with_dict_categorized.head()

,NAID,naraURL,pageObjectId,pageURL,transcriptionText,file_cat,frequency_distribution,frequency_keys,frequency_categories
14,111769405,https://catalog.archives.gov/id/111769405,111769406,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,"(3 - 1776)\nSERVICE\nMass.\nWoodbury, Peter\n1...",soldier,"{'3': 1, '1776': 1, 'service': 1, 'mass': 1, '...",3||1776||service||mass||woodbury||peter||10009...,None
15,111769405,https://catalog.archives.gov/id/111769405,111769407,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Vermont 18.682\n---\nPeter Woodbury\nof Windso...,soldier,"{'vermont': 3, '18.682': 1, '--': 1, 'peter': ...",vermont||18.682||--||peter||woodbury||windsor|...,None
16,111769405,https://catalog.archives.gov/id/111769405,111769408,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,9316\nPeter Woodbury\nAdmit\n11 mos 28 ds\nRet...,soldier,"{'9316': 1, 'peter': 1, 'woodbury': 1, 'admit'...",9316||peter||woodbury||admit||11||mos||28||ds|...,None
17,111769405,https://catalog.archives.gov/id/111769405,111769409,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,BRIEF objections to the admission of Pension C...,soldier,"{'brief': 1, 'objections': 1, 'admission': 1, ...",brief||objections||admission||pension||claims|...,None
18,111769405,https://catalog.archives.gov/id/111769405,111769410,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,2\nXI. Has the applicant obtained the evidence...,soldier,"{'2': 1, 'xi': 1, 'applicant': 5, 'obtained': ...",2||xi||applicant||obtained||evidence||clergyma...,religion


#### Filter frequency_keys column to only include words in the top 10 lists

In [40]:
def filter_frequency_keys(frequency_keys_str):
    """
    Filter frequency_keys to only include words from hopelessness_words, 
    religion_words, and animals_words.
    Returns a string of filtered words separated by '||'
    """
    if pd.isna(frequency_keys_str) or frequency_keys_str == '':
        return ''
    
    # Split the frequency_keys string into individual words
    words = frequency_keys_str.split('||')
    
    # Combine all three word lists into a single set for efficient lookup
    all_category_words = set(hopelessness_words + religion_words + animals_words)
    
    # Filter to only keep words that are in any of the category word lists
    filtered_words = [word for word in words if word in all_category_words]
    
    # Join filtered words with '||', return empty string if none found
    return '||'.join(filtered_words) if filtered_words else ''

In [41]:
# Apply the function to update the frequency_keys column
df_with_dict_categorized['frequency_keys'] = df_with_dict_categorized['frequency_keys'].apply(filter_frequency_keys)

In [42]:
# Check the results
print(f"Total rows: {len(df_with_dict_categorized)}")
print(f"\nSample of filtered frequency_keys:")
print(df_with_dict_categorized[['frequency_keys', 'frequency_categories']].head(20))
print(f"\nRows with non-empty frequency_keys after filtering:")
print(df_with_dict_categorized[df_with_dict_categorized['frequency_keys'] != ''][['frequency_keys', 'frequency_categories']].head(20))

Total rows: 604716

Sample of filtered frequency_keys:
   frequency_keys frequency_categories
14                                None
15                                None
16                                None
17                                None
18      clergyman             religion
19                                None
20                                None
21                                None
22      clergyman             religion
23                                None
24                                None
25                                None
44                                None
45                                None
46                                None
47                                None
48                                None
49                                None
50                                None
51                                None

Rows with non-empty frequency_keys after filtering:
                         frequency_keys frequency_categories
18          

In [44]:
df_with_dict_categorized.shape

(604716, 9)

## get sample rows to save as CSV to use on frontend
##### probably want approximately 100 images per word

#### start by getting rows that contain multiple catetories

In [49]:
# Filter to only rows with multiple categories (contains "||")
df_with_dict_categorized_multi = df_with_dict_categorized[
    df_with_dict_categorized['frequency_categories'].apply(
        lambda x: isinstance(x, str) and '||' in x
    )
].copy()

In [50]:
df_with_dict_categorized_multi.head()

,NAID,naraURL,pageObjectId,pageURL,transcriptionText,file_cat,frequency_distribution,frequency_keys,frequency_categories
3257,111766040,https://catalog.archives.gov/id/111766040,111766046,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,application for a pension which was not grante...,soldier,"{'application': 3, 'pension': 1, 'granted': 1,...",colt||cow||sheep||bible||mare||cows||hogs,religion||animals
5947,111763255,https://catalog.archives.gov/id/111763255,111763275,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,"within the provisions of an act entitled ""an a...",widow,"{'within': 1, 'provisions': 1, 'act': 2, 'enti...",pain||mare||cows||hogs,hopelessness||animals
7096,111762056,https://catalog.archives.gov/id/111762056,111762083,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,An Appeal from the first Congregational Church...,soldier,"{'appeal': 2, 'first': 1, 'congregational': 1,...",church||lord||minister||pastor||god||painful||...,hopelessness||religion
9663,111759450,https://catalog.archives.gov/id/111759450,111759465,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Schedule of property belonging to the Deponent...,soldier,"{'schedule': 1, 'property': 2, 'belonging': 1,...",cow||mare||pigs||suffer,hopelessness||animals
9964,111759108,https://catalog.archives.gov/id/111759108,111759143,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,And I the said Philip Wolfenbarger do furtherm...,widow,"{'said': 5, 'philip': 3, 'wolfenbarger': 2, 'f...",suffering||lord,hopelessness||religion


In [60]:
# Check the results
print(f"Total rows with multiple categories: {len(df_with_dict_categorized_multi)}")
print(f"\nCategory distribution:")
print(df_with_dict_categorized_multi['frequency_categories'].value_counts())
# print(f"\nSample rows:")
# print(df_with_dict_categorized_multi[['frequency_keys', 'frequency_categories']].head(20))

Total rows with multiple categories: 5808

Category distribution:
frequency_categories
religion                           2966
religion||animals                   842
hopelessness||religion              812
animals                             522
hopelessness||animals               437
hopelessness                        171
hopelessness||religion||animals      58
Name: count, dtype: int64


### Check if filtered df has at least 100 entries for each word

In [61]:
# Initialize a Counter to collect all words
multi_category_word_freq = Counter()

# Iterate through each row's frequency_keys
for frequency_keys_str in df_with_dict_categorized_multi['frequency_keys']:
    # Skip empty strings or NaN values
    if pd.isna(frequency_keys_str) or frequency_keys_str == '':
        continue
    
    # Split by '||' to get individual words
    words = frequency_keys_str.split('||')
    
    # Add each word to the counter
    multi_category_word_freq.update(words)

# Convert to regular dictionary and sort by frequency (descending)
multi_category_word_freq_dict = dict(sorted(
    multi_category_word_freq.items(), 
    key=lambda x: x[1], 
    reverse=True
))

# Display results
print(f"Total unique words: {len(multi_category_word_freq_dict)}")
print(f"\nTop 20 most frequent words:")
for word, count in list(multi_category_word_freq_dict.items())[:20]:
    print(f"  {word}: {count}")

# Show the full dictionary
multi_category_word_freq_dict

Total unique words: 40

Top 20 most frequent words:
  church: 2414
  pastor: 1700
  minister: 1224
  lord: 1173
  clergyman: 758
  cow: 641
  sheep: 504
  bible: 477
  cows: 394
  heifer: 365
  colt: 341
  hogs: 341
  afflicted: 322
  cattle: 313
  mare: 312
  bishop: 277
  god: 241
  christian: 230
  pigs: 228
  bull: 218


{'church': 2414,
 'pastor': 1700,
 'minister': 1224,
 'lord': 1173,
 'clergyman': 758,
 'cow': 641,
 'sheep': 504,
 'bible': 477,
 'cows': 394,
 'heifer': 365,
 'colt': 341,
 'hogs': 341,
 'afflicted': 322,
 'cattle': 313,
 'mare': 312,
 'bishop': 277,
 'god': 241,
 'christian': 230,
 'pigs': 228,
 'bull': 218,
 'destitute': 216,
 'poverty': 188,
 'lamb': 171,
 'pain': 167,
 'helpless': 165,
 'pope': 164,
 'hardships': 160,
 'suffering': 159,
 'lambs': 122,
 'needy': 110,
 'hog': 107,
 'distress': 100,
 'suffer': 81,
 'affliction': 48,
 'painful': 47,
 'heifers': 44,
 'pig': 40,
 'hardship': 36,
 'distressed': 31,
 'despondent': 25}

In [62]:
# Get all category words
all_category_words = set(hopelessness_words + religion_words + animals_words)

# Find words that don't have at least 100 entries
words_below_100 = []

for word in all_category_words:
    count = multi_category_word_freq_dict.get(word, 0)
    if count < 100:
        words_below_100.append((word, count))

# Sort by count (ascending)
words_below_100.sort(key=lambda x: x[1])

print(f"Total category words: {len(all_category_words)}")
print(f"Words with fewer than 100 entries: {len(words_below_100)}\n")

print("Words with fewer than 100 entries:")
for word, count in words_below_100:
    print(f"  {word}: {count}")

# Also show which words are missing entirely (count = 0)
missing_words = [word for word, count in words_below_100 if count == 0]
if missing_words:
    print(f"\nWords completely missing (0 entries): {missing_words}")

Total category words: 40
Words with fewer than 100 entries: 8

Words with fewer than 100 entries:
  despondent: 25
  distressed: 31
  hardship: 36
  pig: 40
  heifers: 44
  painful: 47
  affliction: 48
  suffer: 81


In [ ]:
# words_to_get_more_rows = ["heifer", "distress", "despondent", "hardships", "pope", "pastor", "bishop", "colt", "minister", "pain"]

words_to_get_more_rows = ["despondent"]
# Create a set for efficient lookup
words_to_get_more_rows_set = set(words_to_get_more_rows)

### fetch more rows from df_with_dict_categorized and add to df_with_dict_categorized_multi

In [58]:
# Filter rows that:
# 1. Have "||" in frequency_keys (multiple words) - using string contains, not regex
# 2. Contain at least one word from words_to_get_more_rows
def contains_target_word_and_multiple_keys(frequency_keys_str):
    """
    Check if frequency_keys contains "||" (multiple words) 
    AND contains at least one word from words_to_get_more_rows
    """
    if pd.isna(frequency_keys_str) or frequency_keys_str == '':
        return False
    
    # Check if it contains "||" (multiple words)
    if '||' not in frequency_keys_str:
        return False
    
    # Split by '||' to get individual words
    words = set(frequency_keys_str.split('||'))
    
    # Check if any word from words_to_get_more_rows is in the words
    return bool(words.intersection(words_to_get_more_rows_set))

# Filter the dataframe
additional_rows = df_with_dict_categorized[
    df_with_dict_categorized['frequency_keys'].apply(contains_target_word_and_multiple_keys)
].copy()

print(f"Found {len(additional_rows)} additional rows")
print(f"\nCategory distribution of additional rows:")
print(additional_rows['frequency_categories'].value_counts())

# Check which rows are NOT already in df_with_dict_categorized_multi
# Using index to identify unique rows
existing_indices = set(df_with_dict_categorized_multi.index)
new_rows = additional_rows[~additional_rows.index.isin(existing_indices)].copy()

print(f"\nRows already in df_with_dict_categorized_multi: {len(additional_rows) - len(new_rows)}")
print(f"New rows to add: {len(new_rows)}")

# Show sample of new rows
if len(new_rows) > 0:
    print(f"\nSample of new rows to add:")
    print(new_rows[['frequency_keys', 'frequency_categories']].head(10))
    
    # Add the new rows to df_with_dict_categorized_multi
    df_with_dict_categorized_multi = pd.concat([df_with_dict_categorized_multi, new_rows])
    
    print(f"\nUpdated df_with_dict_categorized_multi shape: {df_with_dict_categorized_multi.shape}")
    print(f"\nUpdated category distribution:")
    print(df_with_dict_categorized_multi['frequency_categories'].value_counts())
else:
    print("\nNo new rows to add - all rows already exist in df_with_dict_categorized_multi")

Found 4206 additional rows

Category distribution of additional rows:
frequency_categories
religion                           2966
animals                             522
hopelessness||religion              275
hopelessness                        171
religion||animals                   139
hopelessness||animals               110
hopelessness||religion||animals      23
Name: count, dtype: int64

Rows already in df_with_dict_categorized_multi: 547
New rows to add: 3659

Sample of new rows to add:
                              frequency_keys frequency_categories
53                     cows||heifer||heifers              animals
83                              cows||heifer              animals
3265                cows||colt||sheep||lambs              animals
5852                        church||minister             religion
7077                          pastor||church             religion
8128   cows||heifer||hogs||pigs||sheep||colt              animals
9722                         pain||suf

#### last remaining word to fill up is "despondent" - fetch 75 additional rows

In [64]:
# Filter rows that contain "despondent"
def contains_despondent(frequency_keys_str):
    """Check if frequency_keys contains 'despondent'"""
    if pd.isna(frequency_keys_str) or frequency_keys_str == '':
        return False
    words = set(frequency_keys_str.split('||'))
    return 'despondent' in words

# Get all rows containing "despondent"
despondent_rows = df_with_dict_categorized[
    df_with_dict_categorized['frequency_keys'].apply(contains_despondent)
].copy()

print(f"Found {len(despondent_rows)} total rows containing 'despondent'")

# Check which are NOT already in df_with_dict_categorized_multi
existing_indices = set(df_with_dict_categorized_multi.index)
new_despondent_rows = despondent_rows[~despondent_rows.index.isin(existing_indices)].copy()

print(f"Rows already in df_with_dict_categorized_multi: {len(despondent_rows) - len(new_despondent_rows)}")
print(f"New rows available: {len(new_despondent_rows)}")

# Sort by whether they have multiple keys (favoring rows with "||")
# Create a column to indicate multiple keys
new_despondent_rows['has_multiple_keys'] = new_despondent_rows['frequency_keys'].apply(
    lambda x: '||' in str(x) if pd.notna(x) else False
)

# Sort: first by has_multiple_keys (descending - True first)
new_despondent_rows_sorted = new_despondent_rows.sort_values(
    by='has_multiple_keys',
    ascending=False
)

# Take up to 75 rows
rows_to_add = new_despondent_rows_sorted.head(75).copy()

# Drop the temporary column
rows_to_add = rows_to_add.drop(columns=['has_multiple_keys'])

print(f"\nSelected {len(rows_to_add)} rows to add")
print(f"Rows with multiple keys: {rows_to_add['frequency_keys'].str.contains('||', regex=False, na=False).sum()}")

# Show sample
print(f"\nSample of rows to add:")
print(rows_to_add[['frequency_keys', 'frequency_categories']].head(10))

# Add to df_with_dict_categorized_multi
df_with_dict_categorized_multi = pd.concat([df_with_dict_categorized_multi, rows_to_add])

print(f"\nUpdated df_with_dict_categorized_multi shape: {df_with_dict_categorized_multi.shape}")
print(f"\nUpdated category distribution:")
print(df_with_dict_categorized_multi['frequency_categories'].value_counts())

Found 145 total rows containing 'despondent'
Rows already in df_with_dict_categorized_multi: 25
New rows available: 120

Selected 75 rows to add
Rows with multiple keys: 0

Sample of rows to add:
        frequency_keys frequency_categories
44290       despondent         hopelessness
63525       despondent         hopelessness
1813023     despondent         hopelessness
1803783     despondent         hopelessness
1791612     despondent         hopelessness
1740552     despondent         hopelessness
1740501     despondent         hopelessness
1698597     despondent         hopelessness
1692084     despondent         hopelessness
1616929     despondent         hopelessness

Updated df_with_dict_categorized_multi shape: (5883, 9)

Updated category distribution:
frequency_categories
religion                           2966
religion||animals                   842
hopelessness||religion              812
animals                             522
hopelessness||animals               437
hopelessne

In [66]:
df_with_dict_categorized_multi.shape

(5883, 9)

## now let's see how many unique NAIDs are included in this selection

In [67]:
# Count unique NAID values
naid_counts = df_with_dict_categorized_multi['NAID'].value_counts().to_dict()

# Or using Counter
# naid_counts = dict(Counter(df_with_dict_categorized_multi['NAID']))

print(f"Total unique NAIDs: {len(naid_counts)}")
print(f"Total rows: {len(df_with_dict_categorized_multi)}")

# Show some statistics
print(f"\nNAIDs appearing most frequently:")
sorted_naid_counts = dict(sorted(naid_counts.items(), key=lambda x: x[1], reverse=True))
for naid, count in list(sorted_naid_counts.items())[:10]:
    print(f"  NAID {naid}: {count} times")

# Show NAIDs that appear multiple times
multiple_occurrences = {naid: count for naid, count in naid_counts.items() if count > 1}
print(f"\nNAIDs appearing more than once: {len(multiple_occurrences)}")
if len(multiple_occurrences) > 0:
    print(f"Max occurrences for a single NAID: {max(naid_counts.values())}")

# Display the dictionary
naid_counts

Total unique NAIDs: 3818
Total rows: 5883

NAIDs appearing most frequently:
  NAID 144224657: 11 times
  NAID 53899621: 10 times
  NAID 53929009: 10 times
  NAID 54930281: 9 times
  NAID 54740419: 9 times
  NAID 144225574: 8 times
  NAID 54729136: 8 times
  NAID 196341994: 8 times
  NAID 144075059: 8 times
  NAID 54241007: 8 times

NAIDs appearing more than once: 1212
Max occurrences for a single NAID: 11


{'144224657': 11,
 '53899621': 10,
 '53929009': 10,
 '54930281': 9,
 '54740419': 9,
 '144225574': 8,
 '54729136': 8,
 '196341994': 8,
 '144075059': 8,
 '54241007': 8,
 '53973551': 7,
 '54213872': 7,
 '54937647': 7,
 '54416514': 7,
 '111445353': 7,
 '53974810': 7,
 '144124253': 7,
 '196631399': 7,
 '111781503': 6,
 '53974434': 6,
 '54948446': 6,
 '53973443': 6,
 '54466352': 6,
 '196372017': 6,
 '144300461': 6,
 '54516759': 6,
 '144157311': 6,
 '53906156': 6,
 '54903523': 6,
 '54718724': 6,
 '53923177': 6,
 '196689884': 6,
 '53929423': 6,
 '54604300': 6,
 '144342301': 6,
 '54703036': 6,
 '196305198': 6,
 '144214085': 6,
 '54774383': 6,
 '53914070': 6,
 '54379521': 6,
 '111473065': 6,
 '54072909': 6,
 '196387762': 6,
 '111717203': 6,
 '54939123': 6,
 '53921206': 6,
 '53901727': 6,
 '144046793': 5,
 '111471442': 5,
 '111488655': 5,
 '196182947': 5,
 '54338420': 5,
 '54067449': 5,
 '53829850': 5,
 '54273284': 5,
 '144196901': 5,
 '54053320': 5,
 '54917171': 5,
 '54039443': 5,
 '54070323': 5

### Filter df_with_dict_categorized_multi to maintain the fewest possible NAIDs while still maintaining 100 entries for each word

In [71]:
from collections import Counter, defaultdict

# Define word groups (singular/plural pairs that should be combined)
word_groups = {
    'pig_group': ['pig', 'pigs'],
    'cow_group': ['cow', 'cows'],
    'lamb_group': ['lamb', 'lambs'],
    'heifer_group': ['heifer', 'heifers'],
    'hog_group': ['hog', 'hogs'],
    "afflicted_group": ["afflicted", "affliction"],
    "pain_group": ["pain", "painful"],
    "hardship_group": ["hardship", "hardships"],
    "distress_group": ["distress", "distressed"],
    "suffering_group": ["suffering", "suffer"]
}

# Create a mapping from individual words to their group
word_to_group = {}
for group_name, words in word_groups.items():
    for word in words:
        word_to_group[word] = group_name

# Get all category words
all_category_words = set(hopelessness_words + religion_words + animals_words)

def get_word_group_counts_for_row(frequency_keys_str):
    """Get word group counts for a single row"""
    if pd.isna(frequency_keys_str) or frequency_keys_str == '':
        return {}
    
    words = frequency_keys_str.split('||')
    row_group_counts = defaultdict(int)
    
    for word in words:
        if word in word_to_group:
            row_group_counts[word_to_group[word]] += 1
        elif word in all_category_words:
            row_group_counts[word] += 1
    
    return dict(row_group_counts)

def calculate_group_counts(df):
    """Calculate word group counts for a dataframe"""
    group_counts = defaultdict(int)
    for frequency_keys_str in df['frequency_keys']:
        if pd.isna(frequency_keys_str) or frequency_keys_str == '':
            continue
        row_groups = get_word_group_counts_for_row(frequency_keys_str)
        for group, count in row_groups.items():
            group_counts[group] += count
    return dict(group_counts)

def can_remove_naid(naid, current_df, naid_to_rows):
    """Check if removing this NAID would keep all groups >= 100"""
    # Get rows for this NAID
    rows_to_remove = naid_to_rows[naid]
    
    # Calculate what group counts would be after removal
    test_df = current_df.drop(rows_to_remove)
    test_group_counts = calculate_group_counts(test_df)
    
    # Check if all groups still have >= 100
    return all(count >= 100 for count in test_group_counts.values())

# Start with all NAIDs
all_naids = set(df_with_dict_categorized_multi['NAID'].unique())
selected_naids = all_naids.copy()

# Group rows by NAID
naid_to_rows = defaultdict(list)
for idx, row in df_with_dict_categorized_multi.iterrows():
    naid_to_rows[row['NAID']].append(idx)

# Initial group counts
current_df = df_with_dict_categorized_multi.copy()
current_group_counts = calculate_group_counts(current_df)

print(f"Starting with {len(selected_naids)} NAIDs")
print(f"Initial group counts:")
for group, count in sorted(current_group_counts.items(), key=lambda x: x[1]):
    print(f"  {group}: {count}")

# Iteratively remove NAIDs that aren't needed
removed_count = 0
iteration = 0
max_iterations = len(all_naids)  # Safety limit

# Convert to list for iteration (we'll modify selected_naids as we go)
naids_to_check = list(selected_naids)

for naid in naids_to_check:
    if naid not in selected_naids:
        continue
        
    iteration += 1
    if iteration % 100 == 0:
        print(f"Checking NAID {iteration}/{len(naids_to_check)}: {len(selected_naids)} remaining")
    
    # Check if we can remove this NAID
    if can_remove_naid(naid, current_df, naid_to_rows):
        # Remove this NAID
        rows_to_remove = naid_to_rows[naid]
        current_df = current_df.drop(rows_to_remove)
        selected_naids.remove(naid)
        removed_count += 1

print(f"\nRemoved {removed_count} NAIDs")
print(f"Final selected NAIDs: {len(selected_naids)}")

# Create filtered dataframe with only selected NAIDs
df_with_dict_categorized_multi_reduced = df_with_dict_categorized_multi[
    df_with_dict_categorized_multi['NAID'].isin(selected_naids)
].copy()

print(f"\nReduced dataframe shape: {df_with_dict_categorized_multi_reduced.shape}")
print(f"Original unique NAIDs: {len(all_naids)}")
print(f"Reduced unique NAIDs: {len(selected_naids)}")
print(f"Reduction: {removed_count} fewer NAIDs")

# Verify final word group counts
final_group_counts = calculate_group_counts(df_with_dict_categorized_multi_reduced)

print(f"\nFinal word group counts:")
for group, count in sorted(final_group_counts.items(), key=lambda x: x[1]):
    status = "✓" if count >= 100 else f"✗ (needs {100 - count} more)"
    print(f"  {group}: {count} {status}")

Starting with 3818 NAIDs
Initial group counts:
  despondent: 100
  needy: 110
  distress_group: 131
  pope: 164
  helpless: 165
  poverty: 188
  hardship_group: 196
  pain_group: 214
  destitute: 216
  bull: 218
  christian: 230
  suffering_group: 240
  god: 241
  pig_group: 268
  bishop: 277
  lamb_group: 293
  mare: 312
  cattle: 313
  colt: 341
  afflicted_group: 370
  heifer_group: 409
  hog_group: 448
  bible: 477
  sheep: 504
  clergyman: 758
  cow_group: 1035
  lord: 1173
  minister: 1224
  pastor: 1700
  church: 2414
Checking NAID 100/3818: 3721 remaining
Checking NAID 200/3818: 3624 remaining
Checking NAID 300/3818: 3527 remaining
Checking NAID 400/3818: 3431 remaining
Checking NAID 500/3818: 3337 remaining
Checking NAID 600/3818: 3247 remaining
Checking NAID 700/3818: 3153 remaining
Checking NAID 800/3818: 3062 remaining
Checking NAID 900/3818: 2967 remaining
Checking NAID 1000/3818: 2871 remaining
Checking NAID 1100/3818: 2775 remaining
Checking NAID 1200/3818: 2680 remainin

In [72]:
df_with_dict_categorized_multi_reduced.shape

(1509, 9)

In [73]:
df_with_dict_categorized_multi_reduced.head()

,NAID,naraURL,pageObjectId,pageURL,transcriptionText,file_cat,frequency_distribution,frequency_keys,frequency_categories
5947,111763255,https://catalog.archives.gov/id/111763255,111763275,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,"within the provisions of an act entitled ""an a...",widow,"{'within': 1, 'provisions': 1, 'act': 2, 'enti...",pain||mare||cows||hogs,hopelessness||animals
7096,111762056,https://catalog.archives.gov/id/111762056,111762083,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,An Appeal from the first Congregational Church...,soldier,"{'appeal': 2, 'first': 1, 'congregational': 1,...",church||lord||minister||pastor||god||painful||...,hopelessness||religion
12708,111756184,https://catalog.archives.gov/id/111756184,111756188,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,OBITUARY.\nCommunicated.\n\nDIED --In this cou...,widow,"{'obituary': 1, 'communicated': 1, 'died': 1, ...",christian||church||god||painful,hopelessness||religion
12770,111756184,https://catalog.archives.gov/id/111756184,111756250,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Virginia Fauquier County to wit:\nJohn Dulin a...,widow,"{'virginia': 3, 'fauquier': 3, 'county': 5, 'w...",god||needy,hopelessness||religion
14119,111754847,https://catalog.archives.gov/id/111754847,111754856,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,DECLARATION\nIn order to obtain the benefit of...,rejected,"{'declaration': 3, 'order': 2, 'obtain': 2, 'b...",bull||clergyman,religion||animals


In [74]:
# Count how many times each NAID appears
naid_counts_reduced = df_with_dict_categorized_multi_reduced['NAID'].value_counts().to_dict()

print(f"Total unique NAIDs in reduced dataframe: {len(naid_counts_reduced)}")
print(f"Total rows: {len(df_with_dict_categorized_multi_reduced)}")

# Show statistics
print(f"\nNAIDs appearing most frequently:")
sorted_naid_counts = dict(sorted(naid_counts_reduced.items(), key=lambda x: x[1], reverse=True))
for naid, count in list(sorted_naid_counts.items())[:20]:
    print(f"  NAID {naid}: {count} times")

# Show distribution
print(f"\nDistribution of NAID occurrences:")
occurrence_counts = {}
for count in naid_counts_reduced.values():
    occurrence_counts[count] = occurrence_counts.get(count, 0) + 1

for num_occurrences in sorted(occurrence_counts.keys()):
    num_naids = occurrence_counts[num_occurrences]
    print(f"  {num_occurrences} occurrence(s): {num_naids} NAID(s)")

# Display the full dictionary
naid_counts_reduced

Total unique NAIDs in reduced dataframe: 878
Total rows: 1509

NAIDs appearing most frequently:
  NAID 53929009: 10 times
  NAID 54729136: 8 times
  NAID 144225574: 8 times
  NAID 111445353: 7 times
  NAID 53973551: 7 times
  NAID 54937647: 7 times
  NAID 196631399: 7 times
  NAID 54213872: 7 times
  NAID 54903523: 6 times
  NAID 54718724: 6 times
  NAID 53974434: 6 times
  NAID 54774383: 6 times
  NAID 144214085: 6 times
  NAID 53901727: 6 times
  NAID 54516759: 6 times
  NAID 196305198: 6 times
  NAID 54604300: 6 times
  NAID 111717203: 6 times
  NAID 53973443: 6 times
  NAID 54466352: 6 times

Distribution of NAID occurrences:
  1 occurrence(s): 534 NAID(s)
  2 occurrence(s): 199 NAID(s)
  3 occurrence(s): 75 NAID(s)
  4 occurrence(s): 32 NAID(s)
  5 occurrence(s): 17 NAID(s)
  6 occurrence(s): 13 NAID(s)
  7 occurrence(s): 5 NAID(s)
  8 occurrence(s): 2 NAID(s)
  10 occurrence(s): 1 NAID(s)


{'53929009': 10,
 '54729136': 8,
 '144225574': 8,
 '111445353': 7,
 '53973551': 7,
 '54937647': 7,
 '196631399': 7,
 '54213872': 7,
 '54903523': 6,
 '54718724': 6,
 '53974434': 6,
 '54774383': 6,
 '144214085': 6,
 '53901727': 6,
 '54516759': 6,
 '196305198': 6,
 '54604300': 6,
 '111717203': 6,
 '53973443': 6,
 '54466352': 6,
 '54939123': 6,
 '54067449': 5,
 '53999582': 5,
 '111467534': 5,
 '196417196': 5,
 '54273284': 5,
 '54027938': 5,
 '111722025': 5,
 '54280912': 5,
 '53829850': 5,
 '196270852': 5,
 '144046793': 5,
 '111471442': 5,
 '54053320': 5,
 '53978145': 5,
 '54565318': 5,
 '54883321': 5,
 '196153170': 5,
 '54948026': 4,
 '54160986': 4,
 '196383472': 4,
 '196106496': 4,
 '54653690': 4,
 '196484985': 4,
 '144035954': 4,
 '111520596': 4,
 '196220732': 4,
 '111670224': 4,
 '144328000': 4,
 '54153555': 4,
 '54331202': 4,
 '54106251': 4,
 '144293796': 4,
 '53846041': 4,
 '196142236': 4,
 '54316755': 4,
 '54496699': 4,
 '54603584': 4,
 '196265263': 4,
 '196281899': 4,
 '196186008': 

### Sort by NAID frequency first (preferring those that occur the most), then by number of keys:

In [78]:
# Sort by NAID frequency first, then by number of keys
df_with_dict_categorized_multi_reduced_sorted = df_with_dict_categorized_multi_reduced.copy()

# Count how many times each NAID appears in the dataframe
naid_counts_in_df = df_with_dict_categorized_multi_reduced_sorted['NAID'].value_counts().to_dict()

# Add column with NAID occurrence count
df_with_dict_categorized_multi_reduced_sorted['naid_count'] = df_with_dict_categorized_multi_reduced_sorted['NAID'].map(naid_counts_in_df)

# Count the number of keys in each row
df_with_dict_categorized_multi_reduced_sorted['num_keys'] = df_with_dict_categorized_multi_reduced_sorted['frequency_keys'].apply(
    lambda x: len(str(x).split('||')) if pd.notna(x) and str(x) != '' else 0
)

# Sort: first by NAID count (descending - most frequent NAIDs first), then by num_keys (descending)
df_with_dict_categorized_multi_reduced_sorted = df_with_dict_categorized_multi_reduced_sorted.sort_values(
    by=['naid_count', 'num_keys'], 
    ascending=[False, False]
)

# Optionally drop the temporary columns
# df_with_dict_categorized_multi_reduced_sorted = df_with_dict_categorized_multi_reduced_sorted.drop(columns=['naid_count', 'num_keys'])

# Display the sorted dataframe
df_with_dict_categorized_multi_reduced_sorted

,NAID,naraURL,pageObjectId,pageURL,transcriptionText,file_cat,frequency_distribution,frequency_keys,frequency_categories,naid_count,num_keys
1921635,53929009,https://catalog.archives.gov/id/53929009,53929020,https://s3.amazonaws.com/NARAprodstorage/opast...,This certifies that in the Records of the\nChu...,widow,"{'certifies': 1, 'records': 1, 'church': 4, 'f...",church||minister||pastor||destitute,hopelessness||religion,10,4
1921636,53929009,https://catalog.archives.gov/id/53929009,53929020,https://s3.amazonaws.com/NARAprodstorage/opast...,This certifies that in the Records of the\nChu...,widow,"{'certifies': 1, 'records': 1, 'church': 4, 'f...",church||minister||pastor||destitute,hopelessness||religion,10,4
1921725,53929009,https://catalog.archives.gov/id/53929009,53929065,https://s3.amazonaws.com/NARAprodstorage/opast...,Huntington Sept 11th. '44\nThis certifies that...,widow,"{'huntington': 5, 'sept': 1, '11th': 2, ''44':...",church||minister||pastor,religion,10,3
1921726,53929009,https://catalog.archives.gov/id/53929009,53929065,https://s3.amazonaws.com/NARAprodstorage/opast...,Huntington Sept 11th. '44\nThis certifies that...,widow,"{'huntington': 5, 'sept': 1, '11th': 2, ''44':...",church||minister||pastor,religion,10,3
1921625,53929009,https://catalog.archives.gov/id/53929009,53929015,https://s3.amazonaws.com/NARAprodstorage/opast...,The majors name was Shipman and the adjutant w...,widow,"{'majors': 1, 'name': 2, 'shipman': 1, 'adjuta...",bible||bull,religion||animals,10,2
...,...,...,...,...,...,...,...,...,...,...,...
1179564,54256430,https://catalog.archives.gov/id/54256430,54256445,https://s3.amazonaws.com/NARAprodstorage/opast...,State of New York\nClinton County ss on this 4...,widow,"{'state': 5, 'new': 2, 'york': 2, 'clinton': 1...",despondent,hopelessness,1,1
486924,54492043,https://catalog.archives.gov/id/54492043,54492048,https://s3.amazonaws.com/NARAprodstorage/opast...,Paulding and Col. McCloughey and General Georg...,soldier,"{'paulding': 2, 'col.': 2, 'mccloughey': 1, 'g...",despondent,hopelessness,1,1
355660,196212998,https://catalog.archives.gov/id/196212998,196213005,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,State of New York\nColumbia County\nIs on this...,soldier,"{'state': 3, 'new': 1, 'york': 1, 'columbia': ...",despondent,hopelessness,1,1
310986,54529440,https://catalog.archives.gov/id/54529440,54529443,https://s3.amazonaws.com/NARAprodstorage/opast...,aforesaid under the following named officers A...,rejected,"{'aforesaid': 1, 'following': 1, 'named': 1, '...",despondent,hopelessness,1,1


In [79]:
# Remove the unnecessary columns
df_with_dict_categorized_multi_reduced_sorted_clean = df_with_dict_categorized_multi_reduced_sorted.drop(
    columns=['frequency_distribution', 'naid_count', 'num_keys', 'pageObjectId']
)

# Display the cleaned dataframe
df_with_dict_categorized_multi_reduced_sorted_clean

,NAID,naraURL,pageURL,transcriptionText,file_cat,frequency_keys,frequency_categories
1921635,53929009,https://catalog.archives.gov/id/53929009,https://s3.amazonaws.com/NARAprodstorage/opast...,This certifies that in the Records of the\nChu...,widow,church||minister||pastor||destitute,hopelessness||religion
1921636,53929009,https://catalog.archives.gov/id/53929009,https://s3.amazonaws.com/NARAprodstorage/opast...,This certifies that in the Records of the\nChu...,widow,church||minister||pastor||destitute,hopelessness||religion
1921725,53929009,https://catalog.archives.gov/id/53929009,https://s3.amazonaws.com/NARAprodstorage/opast...,Huntington Sept 11th. '44\nThis certifies that...,widow,church||minister||pastor,religion
1921726,53929009,https://catalog.archives.gov/id/53929009,https://s3.amazonaws.com/NARAprodstorage/opast...,Huntington Sept 11th. '44\nThis certifies that...,widow,church||minister||pastor,religion
1921625,53929009,https://catalog.archives.gov/id/53929009,https://s3.amazonaws.com/NARAprodstorage/opast...,The majors name was Shipman and the adjutant w...,widow,bible||bull,religion||animals
...,...,...,...,...,...,...,...
1179564,54256430,https://catalog.archives.gov/id/54256430,https://s3.amazonaws.com/NARAprodstorage/opast...,State of New York\nClinton County ss on this 4...,widow,despondent,hopelessness
486924,54492043,https://catalog.archives.gov/id/54492043,https://s3.amazonaws.com/NARAprodstorage/opast...,Paulding and Col. McCloughey and General Georg...,soldier,despondent,hopelessness
355660,196212998,https://catalog.archives.gov/id/196212998,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,State of New York\nColumbia County\nIs on this...,soldier,despondent,hopelessness
310986,54529440,https://catalog.archives.gov/id/54529440,https://s3.amazonaws.com/NARAprodstorage/opast...,aforesaid under the following named officers A...,rejected,despondent,hopelessness


In [80]:
# Save to CSV
df_with_dict_categorized_multi_reduced_sorted_clean.to_csv(
    'df_with_dict_categorized_multi_reduced_sorted.csv', 
    index=False
)

In [83]:
import json

# Load the current JSON file
with open('frequency_categories_combined.json', 'r') as f:
    combined_dict = json.load(f)

# Define word groups (singular/plural pairs)
word_groups = {
    'animals': [
        ['cow', 'cows'],
        ['lamb', 'lambs'],
        ['hog', 'hogs'],
        ['pig', 'pigs'],
        ['heifer', 'heifers']
    ],
    'hopelessness': [
        ['afflicted', 'affliction'],
        ['suffering', 'suffer'],
        ['pain', 'painful'],
        ['hardships', 'hardship'],
        ['distress', 'distressed']
    ],
    'religion': []  # No pairs in religion category
}

# Transform each category
transformed_dict = {}

for category, word_dict in combined_dict.items():
    transformed_dict[category] = {}
    
    # Get the word groups for this category
    groups = word_groups.get(category, [])
    
    # Track which words have been processed
    processed_words = set()
    
    # Process word groups
    for group in groups:
        # Find which words in the group exist in the data
        existing_words = [(word, word_dict.get(word, {}).get('count', 0)) 
                         for word in group if word in word_dict]
        
        if len(existing_words) > 1:
            # Sort by count (descending) to get the highest count first
            existing_words.sort(key=lambda x: x[1], reverse=True)
            
            # Use the word with highest count as the key
            main_word = existing_words[0][0]
            
            # Combine all words and counts
            total_count = sum(count for _, count in existing_words)
            all_words = [word for word, _ in existing_words]
            
            transformed_dict[category][main_word] = {
                "count": total_count,
                "words": all_words
            }
            
            # Mark all words in this group as processed
            processed_words.update(group)
    
    # Add remaining words that weren't part of any group
    for word, data in word_dict.items():
        if word not in processed_words:
            transformed_dict[category][word] = data.copy()

# Save the transformed dictionary
with open('frequency_categories_combined.json', 'w') as f:
    json.dump(transformed_dict, f, indent=2)

print("Word groups combined successfully!")
print("\nCombined groups:")
for category, groups in word_groups.items():
    if groups:
        print(f"\n{category}:")
        for group in groups:
            existing = [w for w in group if w in combined_dict.get(category, {})]
            if len(existing) > 1:
                counts = {w: combined_dict[category][w].get('count', 0) for w in existing}
                main = max(existing, key=lambda w: counts[w])
                total = sum(counts.values())
                print(f"  {existing} -> '{main}' (total count: {total})")

Word groups combined successfully!

Combined groups:

animals:
  ['cow', 'cows'] -> 'cow' (total count: 3396)
  ['lamb', 'lambs'] -> 'lamb' (total count: 1891)
  ['hog', 'hogs'] -> 'hogs' (total count: 1434)
  ['pig', 'pigs'] -> 'pigs' (total count: 811)
  ['heifer', 'heifers'] -> 'heifer' (total count: 558)

hopelessness:
  ['afflicted', 'affliction'] -> 'afflicted' (total count: 1190)
  ['suffering', 'suffer'] -> 'suffering' (total count: 941)
  ['pain', 'painful'] -> 'pain' (total count: 706)
  ['hardships', 'hardship'] -> 'hardships' (total count: 718)
  ['distress', 'distressed'] -> 'distress' (total count: 404)
